## Running the algorithm in an azure environment

In [1]:
import pandas as pd
import numpy as np
from Helper_Functions import job, get_batch, getBreakdown_Total

import os
from multiprocessing import Pool
#import warnings
#warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.3f' % x)   
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
from azureml.core import Workspace, Dataset

subscription_id = 'xxxxx'
resource_group = 'xxx'
workspace_name = 'SXXX'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='ysysyys')
data1=dataset.to_pandas_dataframe()
data1.reset_index(drop=True,inplace=True)
data1.head()

,TRAN_DATE,TRAN_ID,TRAN_AMT,TRAN_PARTICULAR,Acct_ID,transaction_type
0,2021-10-15,2628977109232,60000.000,FBNMOBILE ADENIYI OWOLABI ABUDU ACCOMMODATION,Account_0,C
1,2021-10-15,2628977109296,200000.000,WEB APP TO STANBICIBTC BANK OLUWASEUN ALFRED ADE,Account_1,C
2,2021-10-15,2628977109232,20000.000,USSD UNION TRANSFER FROM ASIBE IFEANYI FELIX,Account_2,C
3,2021-10-15,2628977109296,90000.000,TUD IBG UTO JINJIRI MAI ATAMFA SANI SULEIMA,Account_3,C
4,2021-10-15,2628977109232,30000.000,TRANSFER FROM TO AMIDU OPEYEMI TIAMIYU,Account_4,C


In [4]:
data1.isna().sum()

TRAN_DATE           0
TRAN_ID             0
TRAN_AMT            0
TRAN_PARTICULAR     0
Acct_ID             0
transaction_type    0
dtype: int64

In [5]:
%%time
data1['TRAN_AMT'] = data1.TRAN_AMT.astype(float)
data1['TRAN_AMT'] = data1.TRAN_AMT.fillna(0)

CPU times: user 139 ms, sys: 89.7 ms, total: 228 ms
Wall time: 227 ms


In [6]:
def AppRun(df, n_cores, func):
    if __name__ == '__main__':
        pool = Pool(n_cores)
        batch_lst = get_batch(df, n_cores)
        print('Batch created')
        results = pool.map(func, batch_lst)
        print('Result gotten')
        pool.close()
        print('pool closed')
        pool.join()
        print('pool joined')
        results_parallel = pd.concat(results)
        print('final result combined')
    
    return results_parallel

In [7]:
n_cores = os.cpu_count()

In [9]:
%%time

result_parallel = AppRun(data1, n_cores-1, job)

Batch created
Result gotten
pool closed
pool joined
final result combined
CPU times: user 26.3 s, sys: 2.59 s, total: 28.9 s
Wall time: 14min 45s


## sample_review

In [10]:
result_parallel.Label.value_counts()

Non Salaried          535023
Salaried              204587
Pensioner              16089
Salaried-Pensioner      2728
Name: Label, dtype: int64

In [11]:
result_parallel.inflowPrevMonth.value_counts()

False    460239
True     298188
Name: inflowPrevMonth, dtype: int64

In [12]:
result_parallel[result_parallel['Acct_ID']=='Account_1663']

,Acct_ID,Label,Duration,Avg Salary Inflow,Avg Pension Inflow,Avg Inflow,Trans Group,inflowCount,inflowPrevMonth,Record_count,possibleGroups
7369,Account_1663,Non Salaried,2021-10-27 00:00:00 - 2022-02-11 00:00:00,nan,nan,99416.667,90000-100000,nan,False,95,0


In [13]:
result_parallel.to_csv("results.csv",sep='\t', index=False)

In [14]:
temp.Label.value_counts()

Salaried              101
Non Salaried           41
Salaried-Pensioner      2
Name: Label, dtype: int64

In [ ]:
# breakdown = getBreakdown_Total(data1)
# breakdown.head()